In [21]:
#pip install evalml

In [2]:
import pandas as pd
import numpy as np
train = pd.read_csv('train_2kmZucJ.csv')
test=pd.read_csv('test_oJQbWVk.csv')
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [3]:
test_n=test.drop('id',axis=1)

In [4]:
#separating input and output features
X=train.drop(['label','id'],axis=1)
Y=train['label']

In [5]:
type(X)

pandas.core.frame.DataFrame

In [6]:
X.head()

,tweet
0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,Finally a transparant silicon case ^^ Thanks t...
2,We love this! Would you go? #talk #makememorie...
3,I'm wired I know I'm George I was made that wa...
4,What amazing service! Apple won't even talk to...


In [7]:
Y.value_counts(normalize=True)

0    0.744192
1    0.255808
Name: label, dtype: float64

In [8]:
import evalml

In [9]:
X_train,X_test,y_train,y_test=evalml.preprocessing.split_data(X,Y,problem_type='binary')

In [10]:
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [11]:
X_train.head()

Data Column,tweet
Physical Type,string
Logical Type,NaturalLanguage
Semantic Tag(s),[]
3050,best day :) #autographs #signatures #sony #mus...
2531,Can't wait for my upgrade - my iPhone is a loa...
4239,Thank u #election2012 for making me use my #sm...
7762,@rakua #Hi & #TFF No #bio ? or #website ? I kn...
979,Suckerrrrr. #self #selfie #suck #lollypop #can...


In [12]:
from evalml import AutoMLSearch

In [13]:
automl=AutoMLSearch(X_train=X_train,y_train=y_train,problem_type='binary',max_batches=1,optimize_thresholds=True)

Generating pipelines to search over...


In [14]:
automl.search()

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: linear_model, extra_trees, catboost, xgboost, decision_tree, random_forest, lightgbm



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 8.836
Batch 1: (2/9) Decision Tree Classifier w/ Text Feat... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.591
High coefficient of variation (cv >= 0.2) within cross validation scores. Decision Tree Classifier w/ Text Featurization Component may not perform as estimated on unseen data.
Batch 1: (3/9) LightGBM Classifier w/ Text Featuriza... Elapsed:00:44
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.334
Batch 1: (4/9) Extra Trees Classifier w/ Text Featur... Elapsed:01:29
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.386
Batch 1: (5/9) Elastic Net Classifier w/ Text Featur... Elapsed:02:20
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.569
Batch 1: (6/9) CatBoost Classifier w/ Text Featuriza..

In [15]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,7,Random Forest Classifier w/ Text Featurization...,0.317864,0.315383,96.402788,False,{'Random Forest Classifier': {'n_estimators': ...
1,6,XGBoost Classifier w/ Text Featurization Compo...,0.322644,0.326897,96.348684,False,"{'XGBoost Classifier': {'eta': 0.1, 'max_depth..."
2,8,Logistic Regression Classifier w/ Text Featuri...,0.329989,0.332837,96.265564,False,{'Logistic Regression Classifier': {'penalty':...
3,2,LightGBM Classifier w/ Text Featurization Comp...,0.333517,0.334886,96.225646,False,{'LightGBM Classifier': {'boosting_type': 'gbd...
4,3,Extra Trees Classifier w/ Text Featurization C...,0.385913,0.375192,95.632683,False,{'Extra Trees Classifier': {'n_estimators': 10...
5,5,CatBoost Classifier w/ Text Featurization Comp...,0.567860,0.567507,93.573620,False,"{'CatBoost Classifier': {'n_estimators': 10, '..."
6,4,Elastic Net Classifier w/ Text Featurization C...,0.568763,0.568623,93.563398,False,"{'Elastic Net Classifier': {'alpha': 0.5, 'l1_..."
7,1,Decision Tree Classifier w/ Text Featurization...,0.590801,0.685063,93.314001,True,{'Decision Tree Classifier': {'criterion': 'gi...
8,0,Mode Baseline Binary Classification Pipeline,8.836388,8.830937,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}


In [16]:
automl.best_pipeline

GeneratedPipeline(parameters={'Random Forest Classifier':{'n_estimators': 100, 'max_depth': 6, 'n_jobs': -1},})

In [17]:
best_pipeline = automl.best_pipeline

In [18]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

************************************************************
* Random Forest Classifier w/ Text Featurization Component *
************************************************************

Problem Type: binary
Model Family: Random Forest

Pipeline Steps
1. Text Featurization Component
2. Random Forest Classifier
	 * n_estimators : 100
	 * max_depth : 6
	 * n_jobs : -1

Training
Training for binary problems.
Total training time (including CV): 48.4 seconds

Cross Validation
----------------
             Log Loss Binary  MCC Binary   AUC  Precision    F1  Balanced Accuracy Binary  Accuracy Binary  Sensitivity at Low Alert Rates # Training # Validation
0                      0.315       0.638 0.917      0.738 0.729                     0.816            0.863                           0.315   4224.000     2112.000
1                      0.325       0.627 0.910      0.711 0.724                     0.817            0.856                           0.286   4224.000     2112.000
2                    

In [19]:
#evaluate on test model
scores = best_pipeline.score(X_test, y_test,  objectives=evalml.objectives.get_core_objectives('binary'))
print(f'Accuracy Binary: {scores["Accuracy Binary"]}')

Accuracy Binary: 0.860479797979798


In [23]:
# from evalml.model_understanding.graphs import confusion_matrix
# y_pred = pipeline.predict(X_test)
# confusion_matrix(y_test, y_pred)

In [24]:
#predict on new test data
pipeline = automl.best_pipeline
pipeline.predict(test_n)

<DataColumn: label (Physical Type = Int64) (Logical Type = Integer) (Semantic Tags = {'numeric'})>

In [26]:
sv=pipeline.predict(test_n)

In [28]:
test

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...
...,...,...
1948,9869,"#SamsungGalaxyNote7 Explodes, Burns 6-Year-Old..."
1949,9870,Now Available - Hoodie. Check it out here - ht...
1950,9871,There goes a crack right across the screen. If...
1951,9872,@codeofinterest as i said #Adobe big time we m...


In [29]:
sv

<DataColumn: label (Physical Type = Int64) (Logical Type = Integer) (Semantic Tags = {'numeric'})>

In [33]:
sv.shape

(1953,)

In [50]:
sv.name

'label'

In [56]:
a=sv.to_series()

In [59]:
a.to_csv('saved_data.csv')